In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import XGBModel
from darts.metrics import mse, rmse, r2_score, mae, smape, wmape
from darts.dataprocessing.transformers import Scaler
import torch
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [9]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [10]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [11]:
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [12]:
from sklearn.model_selection import TimeSeriesSplit

def objective(trial):
    num_target_lags = trial.suggest_int('num_target_lags', 5, 168, step=12)
    target_lags_list = [-i for i in range(1, num_target_lags + 1)]
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 0.5, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 0.5, log=True),
        'n_jobs': -1
    }

    n_splits = 3
    tscv = TimeSeriesSplit(n_splits=n_splits)
    overall_smape_list = []
    fold = 0

    for train_index, val_index in tscv.split(df):
        print(f"\nFold {fold+1}/{n_splits}")
        train_df_fold = df.iloc[train_index]
        val_df_fold = df.iloc[val_index]
        train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
        val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')


        print(f"\nStarting Trial {trial.number}")
        print(f"Hyperparameters: {trial.params}")
        print("\nTraining the model...")
        print(f"Train set: {len(train_fold)} samples")
        print(f"Validation set: {len(val_fold)} samples")

        scaler = Scaler()
        scaler = scaler.fit(train_fold)
        train_scaled = scaler.transform(train_fold)
        val_scaled = scaler.transform(val_fold)

        model = XGBModel(
            lags=target_lags_list,
            output_chunk_length=1,
            add_encoders=encoders,
            **xgb_params
        )

        model.fit(
            series=train_scaled,
            val_series=val_scaled,
            verbose=False,
        )

        forecasts = model.historical_forecasts(
            val_scaled,
            forecast_horizon=24,
            stride=1,
            retrain=False,
            verbose=False
        )

        forecasts_t = scaler.inverse_transform(forecasts)
        s = scaler.inverse_transform(val_scaled)
        metrics = {}
        print("Starting time series verification for consistency...")
        try:
            for target in target_columns:
                metrics[target] = {
                    'MSE': mse(s[target], forecasts_t[target]),
                    'RMSE': rmse(s[target], forecasts_t[target]),
                    'MAE': mae(s[target], forecasts_t[target]),
                    'R2': r2_score(s[target], forecasts_t[target]),
                    'SMAPE': smape(s[target], forecasts_t[target]),
                    'WMAPE': wmape(s[target], forecasts_t[target]),
                }
            metrics_df = pd.DataFrame(metrics).T
            print("\nPerformance metrics:")
            print(metrics_df)
        except Exception as e:
            print(e)
            
        overall_smape_val = smape(val_scaled, forecasts)
        print(f"SMAPE fold {fold}: {overall_smape_val}")
        overall_smape_list.append(overall_smape_val)
        fold += 1

    mean_smape = np.mean(overall_smape_list)
    print(f"Média dos SMAPE nos folds: {mean_smape}")

    trial_dict = {
        "trial_number": trial.number,
        "fold_smape": overall_smape_list,
        "mean_smape": mean_smape,
        "hyperparameters": trial.params
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return mean_smape if not np.isnan(mean_smape) else float("inf")


In [13]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [14]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 10
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-06-17 11:55:49,990] A new study created in memory with name: no-name-dba79bba-0edf-474f-8b3b-0d039d23337b
/home/eduardo/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/optuna/distributions.py:699: UserWarning: The distribution is specified by [5, 168] and step=12, but the range is not divisible by `step`. It will be replaced by [5, 161].
  warnings.warn(
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.



Fold 1/3

Starting Trial 0
Hyperparameters: {'num_target_lags': 137, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.020402363287890857, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.7, 'gamma': 0.05, 'reg_alpha': 2.1505047490246614e-05, 'reg_lambda': 0.001483166190391993}

Training the model...
Train set: 2078 samples
Validation set: 2077 samples


`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                    MSE         RMSE          MAE         R2  \
Temperature                8.397809e+00     2.897897     2.153519   0.566119   
Precipitation_accumulated  1.879731e+05   433.558654   416.916571 -12.273683   
Humidity                   2.515186e+02    15.859339    14.573043  -2.447494   
Wind_Speed_kmh             8.427383e-01     0.918008     0.899911  -4.948751   
Soil_Moisture              2.087452e+06  1444.801884  1374.945027  -2.366269   
Soil_Temperature           1.048012e+01     3.237302     2.416701   0.473872   
Wind_Dir_Sin               1.118815e-01     0.334487     0.162555  -0.143526   
Wind_Dir_Cos               1.570728e+00     1.253287     1.195946  -3.805843   

                                SMAPE       WMAPE  
Temperature                 14.529112   13.508243  
Precipitation_accumulated  182.696399   98.572828  
Humidity                    18.770179   17.19359

`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


...

Starting time series verification for consistency...

Performance metrics:
                                    MSE         RMSE          MAE         R2  \
Temperature                5.968909e+00     2.443135     1.769203   0.684020   
Precipitation_accumulated  1.891483e+05   434.911878   419.503967 -13.373787   
Humidity                   3.393693e+01     5.825541     4.307724   0.505860   
Wind_Speed_kmh             2.173072e-01     0.466162     0.448924  -0.602220   
Soil_Moisture              2.155265e+06  1468.082033  1339.306320  -2.719442   
Soil_Temperature           7.142310e+00     2.672510     1.913761   0.633853   
Wind_Dir_Sin               2.131384e-01     0.461669     0.409941  -1.199442   
Wind_Dir_Cos               1.205813e+00     1.098095     1.047143  -2.862649   

                                SMAPE       WMAPE  
Temperature                 11.843629   11.131144  
Precipitation_accumulated  183.406326   98.575422  
Humidity                     5.187946    5.07318

`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
Specified past encoders in `add_encoders` at model creation but model does not accept past covariates. past encoders will be ignored.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Starting time series verification for consistency...

Performance metrics:
                                     MSE        RMSE         MAE         R2  \
Temperature                     1.319705    1.148784    0.855800   0.495540   
Precipitation_accumulated       6.094063    2.468616    2.399976 -25.132615   
Humidity                       11.381008    3.373575    2.761899   0.703381   
Wind_Speed_kmh                  0.003258    0.057079    0.040412  -1.957877   
Soil_Moisture              225726.490083  475.106820  443.622574  -6.245495   
Soil_Temperature                0.580975    0.762217    0.503179   0.662479   
Wind_Dir_Sin                    0.030578    0.174865    0.075330   0.030624   
Wind_Dir_Cos                    0.046928    0.216630    0.054162   0.015129   

                                SMAPE       WMAPE  
Temperature                 11.199224   10.388396  
Precipitation_accumulated    0.524597    0.523264  
Humidity                     3.196876    3.161530  
Wind_

`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`. To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
[I 2025-06-17 14:03:37,708] Trial 69 finished with value: 55.00346232094776 and parameters: {'num_target_lags': 149, 'n_estimators': 550, 'max_depth': 4, 'learning_rate': 0.09333160505592525, 'min_child_weight': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'gamma': 0.05, 'reg_alpha': 0.0004476502788668159, 'reg_lambda': 0.48241511884894367}. Best is trial 23 with value: 53.90968358560392.


Starting time series verification for consistency...

Performance metrics:
                                     MSE        RMSE         MAE        R2  \
Temperature                    15.105554    3.886586    3.175514  0.473896   
Precipitation_accumulated      48.726347    6.980426    6.652754 -9.907561   
Humidity                       92.738165    9.630066    7.768539  0.355648   
Wind_Speed_kmh                  1.457121    1.207113    0.931161 -0.055019   
Soil_Moisture              227790.174308  477.273689  270.423467  0.433242   
Soil_Temperature               13.333370    3.651489    2.917566  0.228191   
Wind_Dir_Sin                    0.672405    0.820003    0.686336 -0.246498   
Wind_Dir_Cos                    0.413837    0.643301    0.537587 -0.048327   

                                SMAPE      WMAPE  
Temperature                 46.179450  37.989106  
Precipitation_accumulated    1.443182   1.433873  
Humidity                     9.096508   8.856544  
Wind_Speed_kmh    

In [15]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 53.90968358560392
Best Parameters: {'num_target_lags': 149, 'n_estimators': 350, 'max_depth': 4, 'learning_rate': 0.09466638186506064, 'min_child_weight': 10, 'subsample': 0.9, 'colsample_bytree': 0.7, 'gamma': 0.05, 'reg_alpha': 0.0011297380165811975, 'reg_lambda': 0.006714113276880211}


In [16]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
